In [ ]:
#! If you want to force use of CPU (before import of tensorflow)
import os
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
# os.environ['OMP_NUM_THREADS'] = '4'
# os.environ['export OPENBLAS_NUM_THREADS']='4'

#! import as tf
import tensorflow as tf

#! If you want to limit number of threads, the mechine has 32 virtual cores (16 physical ones)
print('thread inter = ', tf.config.threading.get_inter_op_parallelism_threads() )
print('thread intra = ', tf.config.threading.get_intra_op_parallelism_threads() )
 
tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)

#! If you want to limit the GPU memory usage, the machine has ~24GB. For most applications a few GBs is enough.
#! If you max out what you specify progressively up the threshhold.
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_logical_device_configuration(
    physical_devices[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=10_000)])

  logical_devices = tf.config.list_logical_devices('GPU')

except:
  # Invalid device or cannot modify logical devices once initialized.
  pass

#! Will print out what devices you are using
tf.config.list_physical_devices()



import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

from mlkernels.tensorflow import Matern32, EQ, Matern12, Matern52


from LFM.NeuralNet import NeuralN
from LFM.utilities import pshape
from LFM.utilities import reparameterize, log_normal_pdf, expHighLowFunc_01, exp_start_end_x_func
from LFM.GICNet    import FFLayer, KIPLayer, GICNet

pi  = tf.constant(np.pi, tf.float32)
tfd = tfp.distributions
tfb = tfp.bijectors


dirname='data-deeponet-burgers/'
os.makedirs(dirname, exist_ok=True)



In [ ]:
dimU = 1
dimZ = 2
dimW = 1
dimX = 2
                         #  x         t
domain     = tf.constant( [[0., 1.], [0., 1.]] )

# resolution = tf.constant([15,15])
# resolution = tf.constant([30,30])
# resolution = tf.constant([50,50])
resolution = tf.constant([100,100])
nxSamples  = tf.reduce_prod(resolution)
print('resolution = ', resolution)

#currently in dataset
zDist = tfd.Uniform(low=tf.constant([0.01, 0.5], shape=[dimZ]), high=tf.constant([0.1, 1.], shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant([0.5], shape=[dimW]), high=tf.constant([2.], shape=[dimW]))


e_bound = 1e-5

def raveled2DDomainGrid(resolution):
    x, t          = tf.linspace(domain[0][0], domain[0][1], resolution[0]), tf.linspace(domain[1][0], domain[1][1], resolution[1])
    X, T          = np.meshgrid(x, t, indexing='ij')
    Xr, Tr        = tf.reshape(tf.constant(X, dtype=tf.float32), [-1,1]), tf.reshape(tf.constant(T, dtype=tf.float32), [-1,1])
    XTr           = tf.concat([Xr, Tr], 1)
    return XTr

b      = zDist.sample()
gridXTr = raveled2DDomainGrid(resolution)
print('gridXTr.shape', gridXTr.shape)
    
    

In [ ]:
# Define fw network

nhfw             = 400
dimBranchOutfw   = 400
dimTruncOutfw    = 400
dim_DeepO_outfw  = dimU

fwNet  = NeuralN('fw', np.array([ 1, 1, dimZ]) )

fwNet.branch   =  tf.keras.Sequential([ tf.keras.layers.Dense(  nhfw, input_dim=( dimZ + dimW ), activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense( dimBranchOutfw ) ])

fwNet.trunc   = tf.keras.Sequential([   tf.keras.layers.Dense(  nhfw,  input_dim=( dimX ), activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense(  nhfw, activation='swish'),
                                        tf.keras.layers.Dense( dimTruncOutfw ) ])

fwNet.final_bias = tf.Variable( tf.constant(0.), trainable=True)


def fwMap(self, z, w, x):

        z       = tf.reshape(z, [-1, dimZ])
        w       = tf.reshape(w, [-1, dimW])
        x_trunc = tf.reshape(x, [-1, dimX])

        x_trunc  = self.trunc( x_trunc ) # [nxSamples, dimTruncOut]
        x_branch = self.branch( tf.concat([z, w], 1) ) # [1, dimBranchOut]

        print('x_trunc.shape',  x_trunc.shape)
        print('x_branch.shape', x_branch.shape)
        
        mapped = tf.einsum('bi,ni->bn', x_trunc, x_branch ) + self.final_bias

        print('mapped.shape', mapped.shape)

        tf.debugging.assert_all_finite(mapped, 'mapped not finite')
        
        return mapped

from types import MethodType
fwNet.call = MethodType(fwMap, fwNet)
outfw = fwNet(tf.constant(1., shape=[dimZ]), tf.constant(1., shape=[dimW]), gridXTr)

print('outfw.shape', outfw.shape)

fwNet.compile()
fwNet.NN.summary()
fwNet.summary()

# fwNet.save_weights(dirname + '/fwModel/fwModel')
# fwNet.load_weights(dirname + '/fwModel/fwModel')

# del fwNet


In [ ]:

def D_x(x):
    return tf.sin(pi * x)

def initU(x, w):
    return tf.sin(2. * tf.reshape(w, []) * pi * x) * D_x(x)

def D_t(t):
    return t

fwNet.epsilon_r = tf.Variable(1., trainable=True)

def residualFunction( zw ):
    
    # x, t = xrand[:,:1], xrand[:, 1:]
    x, t = gridXTr[:, :1], gridXTr[:, 1:]
    print('zw', zw)
    z, w = zw[ :dimZ ], zw[ dimZ:dimZ+dimW ]
    z    = tf.reshape(z, [-1,1])

    z = tf.reshape(z, [-1])
    w = tf.reshape(w, [ ])

    with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t1:
        t1.watch( x )

        with tf.GradientTape(watch_accessed_variables=False, persistent=True) as t2:
            t2.watch( x )
            t2.watch( t )

            u_untrans = fwNet( z, w, tf.concat([x,t], 1) )

            u = initU(x, w) + D_x(x) * D_t(t) * u_untrans
            
        u_x = tf.reshape(t2.gradient(u, x), [-1,1])
        u_t = tf.reshape(t2.gradient(u, t), [-1,1])

    u_xx = tf.reshape(t1.gradient(u_x, x), [-1,1])
    
    res = u_t +  z[1] * u * u_x  - z[0] * u_xx 
    # res = tf.reduce_mean( res**2. )
    res = tf.reduce_mean( tf.abs(res) )
    tf.debugging.assert_all_finite(res, 'res not finite')
    
    print("residualBody.shape", res.shape)
    print('u.shape', u.shape)

    return res


zs =  zDist.sample( )
ws =  wDist.sample( )

residualFunction( tf.concat([zs, ws], 0) )

In [ ]:


trainableVarsfw   = [ fwNet.trainable_variables ]

@tf.function
def train_step_fw(optimizer, batch):

    with tf.GradientTape(persistent=True) as tape:
    
        lossArray = tf.vectorized_map( residualFunction, batch )
        loss = tf.reduce_mean(lossArray)

    for vars in trainableVarsfw:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss

# batchSize = 25
batchSize = 5

# nData     = 1_000
nData     = 10_000
datasetZ  = zDist.sample( nData )
datasetW  = wDist.sample( nData )
datasetZW = tf.concat([datasetZ, datasetW], 1)
dataset        = tf.data.Dataset.from_tensor_slices( datasetZW )
datasetBatched = dataset.batch(batch_size=batchSize)

num_iterations   = tf.constant(int(2_000_000/(25/batchSize)), dtype=tf.int32)

epochs           = int(num_iterations / nData)

intervalEPOCHsave = 1


lr = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations/batchSize/5), 0.5, staircase=True, name=None
)
optimizer = tf.keras.optimizers.Adam( lr )

lossfw_all     = []
startTrainingfw = time.time()

iteration = tf.constant(0)
for epoch in tf.range( epochs ):
    start_time = time.time()
    for batch in datasetBatched:
        iteration += batchSize
        loss = train_step_fw(optimizer, batch )
    if iteration >= num_iterations: 
        print('DONE BREAK')
        break

    end_time = time.time()

    if epoch % intervalEPOCHsave == 0:
        lossfw_all.append( -loss ) 
        print('Iter: {}, Epoch: {}, Test set loss: {}, time elapse for current epoch: {:.2f}'
           .format(iteration, epoch, -loss, end_time - start_time))

timeTrainfw = (time.time() - startTrainingfw)/60
print('timeTrainfw = {:.2f}m'.format(timeTrainfw))



In [ ]:
lossfw_all = tf.stack(lossfw_all)
LOSS_fw    = np.array(lossfw_all)
plt.plot(range(LOSS_fw.shape[0]), LOSS_fw )
plt.show()
print(np.isnan(LOSS_fw).mean())


In [ ]:
# Define bw network

dvbw             = 20
nh_bw            = 400
dimBranchOutbw   = 400
dimTruncOutbw    = 400
dim_DeepO_outbw  = dimZ

bwNet  = NeuralN('bw', np.array([ 1, 1, dimZ]) )


bwNet.trunc   =  tf.keras.Sequential([  tf.keras.layers.Dense( nh_bw, input_dim=( dimW ), activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( nh_bw, activation='swish'),
                                        tf.keras.layers.Dense( dimTruncOutbw ) ])

bwNet.branch   = tf.keras.Sequential([  tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Conv1D( dvbw, (2), strides=(1), activation='swish' ),
                                        tf.keras.layers.Flatten(  ),
                                        tf.keras.layers.Dense(500),
                                        tf.keras.layers.Dense(dimBranchOutbw) ])

bwNet.final_bias = tf.Variable( tf.constant(0., shape=[dimZ]), trainable=True)

bwNet.lower_alpha = tf.constant(1e-5)
bwNet.upper_w     = tf.constant(1.)

def bwMap(self, u, w, x):

        branch_in = tf.concat([x, u], 1)
        trunc_in  = tf.reshape(w, [-1,dimW])

        x_trunc  = self.trunc( trunc_in ) # [nxSamples, dimTruncOut]
        x_branch = self.branch( branch_in[None, ...] ) # [1, dimBranchOut]

        print('x_trunc.shape',  x_trunc.shape)
        print('x_branch.shape', x_branch.shape)

        x_trunc  = tf.reshape(x_trunc,  [-1, dimZ])
        x_branch = tf.reshape(x_branch, [-1, dimZ])

        print('x_trunc.shape',  x_trunc.shape)
        print('x_branch.shape', x_branch.shape)
        
        mapped = tf.einsum('ij,ij->j', x_trunc, x_branch ) + self.final_bias
        mapped = tf.reshape(mapped, [1,-1])
        
        print('mapped.shape', mapped.shape)

        tf.debugging.assert_all_finite(mapped, 'mapped not finite')
        
        return mapped

from types import MethodType
bwNet.call = MethodType(bwMap, bwNet)
bwNet(tf.constant(1., shape=[nxSamples, dimU]), tf.constant(1., shape=[dimW]), gridXTr)
bwNet.compile()
bwNet.summary()

# bwNet.save_weights(dirname + '/bwModel/bwModel')
# bwNet.load_weights(dirname + '/bwModel/bwModel')

# del bwNet


In [ ]:
def bind(zw):
    z, w = zw[ :dimZ ], zw[ dimZ:dimZ+dimW ]
    return tf.squeeze( fwNet( z, w, gridXTr ))    

datasetUfw = tf.vectorized_map(bind, datasetZW )
print(datasetUfw.shape)

datasetUWZ        = tf.concat([datasetUfw, datasetW, datasetZ], 1)
datasetUWZBatched =  tf.data.Dataset.from_tensor_slices( datasetUWZ ).batch(batch_size=batchSize)

In [ ]:
def bwLoss( u_untranswz ):
    u_untrans, w, z = u_untranswz[ :nxSamples ], u_untranswz[ nxSamples:nxSamples+dimW ], u_untranswz[nxSamples+dimW:]
    print('u_untrans.shape', u_untrans)
    print('w.shape', w)
    print('z.shape', z)
    predbw    = bwNet( tf.reshape(u_untrans, [-1,1]), w , gridXTr)
    loss      = tf.reduce_mean(  (tf.reshape( predbw, [-1] ) - tf.reshape( z, [-1] ))**2. )
    return loss
bwLoss( datasetUWZ[0] )

In [ ]:
trainableVarsbw   = [ bwNet.trainable_variables ]

@tf.function
def train_step_bw(optimizer, batch):

    with tf.GradientTape(persistent=True) as tape:

        losses = tf.vectorized_map( bwLoss, batch )
        loss   = tf.reduce_mean( losses )

    for vars in trainableVarsbw:
        gradients = tape.gradient(loss, vars)
        optimizer.apply_gradients(zip(gradients, vars))

    return loss

num_iterations_bw = num_iterations

intervalEPOCHsave = 1

lr_bw = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3, int(num_iterations_bw/batchSize/5), 0.5, staircase=True, name=None
)
optimizer_bw = tf.keras.optimizers.Adam( lr_bw )

lossAll_bw     = []
startTrainingDeepO_bw = time.time()

iteration_bw = tf.constant(0)
startTrainingbw = time.time()
for epoch in tf.range( epochs ):
    start_time = time.time()
    for batch in datasetUWZBatched:
        iteration_bw += batchSize
        loss = train_step_bw(optimizer_bw, batch)
    if iteration_bw >= num_iterations_bw: 
        print('DONE BREAK')
        break

    end_time = time.time()

    if epoch % intervalEPOCHsave == 0:
        lossAll_bw.append( -loss ) 
        print('Iter: {}, Epoch: {}, Test set loss: {}, time elapse for current epoch: {:.2f}'
           .format(iteration_bw, epoch, -loss, end_time - start_time))


timeTrainbw = (time.time() - startTrainingbw)/60
print('timeTrainbw = {:.2f}m'.format(timeTrainbw))


In [ ]:
lossAll_bw = tf.stack(lossAll_bw)
LOSS_bw    = np.array(lossAll_bw)
plt.plot(range(LOSS_bw.shape[0]), LOSS_bw )
plt.show()
print(np.isnan(LOSS_bw).mean())

In [ ]:
z = tf.constant([0.01, 1.])
w = tf.constant(2.)

# z = [1., 1.]
# w = [1.]

resolutionTEST = [200,200]

X, T = np.meshgrid( tf.linspace(0., 1., resolutionTEST[0]), tf.linspace(0., 1., resolutionTEST[1]), indexing='ij')

xr, tr = tf.reshape(X, [-1,1]), tf.reshape(T, [-1,1])
u_untrans = fwNet( z, w, tf.concat([xr, tr], 1) )

u = initU(xr, w) + D_x(xr) * D_t(tr) * u_untrans

plt.contourf(X, T, tf.reverse(tf.reshape(u, resolutionTEST), [0]), 50)
plt.colorbar()
plt.show()


In [ ]:
dirnameGT = '../../../fenics/Burgers/data-groundTruth-burgers_1000'

print('totTimetraining = ', timeTrainfw + timeTrainbw)

groundTruthU  = tf.constant( np.load(dirnameGT+'/' + 'SolutionsUGT.npy' ), dtype=tf.float32)[:, 1:-1:1,1::1]
allTrueZs     = tf.constant( np.loadtxt(dirnameGT+'/' + 'allZ.dat' ), dtype=tf.float32)
allTrueWs     = tf.constant( np.loadtxt(dirnameGT+'/' + 'allW.dat' ), dtype=tf.float32)
groundTruthXT = tf.constant( np.load(dirnameGT+'/' + 'FenicsMeshXT.npy'     ), dtype=tf.float32)[:, 1:-1:1,1::1]

print('groundTruthU.shape', groundTruthU.shape)
print('allTrueZs.shape', allTrueZs.shape)
print('allTrueWs.shape', allTrueWs.shape)
print('groundTruthXT.shape', groundTruthXT.shape)

nx = groundTruthXT.shape[1]
nt = groundTruthXT.shape[2]




saveDataList = True

if saveDataList:

     allresAbsMean = []

     allPredZs     = []

     allUPreds     = []

     allZNSE       = []
     allUNSE       = []


print('groundTruthXT.shape', groundTruthXT.shape)

Xtest, Ttest = groundTruthXT[0], groundTruthXT[1]

xtest, ttest = tf.reshape(Xtest, [-1,1]), tf.reshape(Ttest, [-1,1])
xttest_rav   = tf.concat([xtest, ttest], 1)

tf_residualFunction = tf.function( residualFunction )#residualFunction #
tf_fwNet            = tf.function( fwNet            )#alphaNet         #
tf_bwNet            = tf.function( bwNet            )#betaNet          #

PLOT = False
savePlots   = False
BenchParams = False
writeToFile = True

kip = KIPLayer(EQ(), 1./tf.cast(resolution[0], tf.float32))

for i in range(groundTruthU.shape[0]):

     
     zTrue = tf.reshape( allTrueZs[i], [-1, 1] )
     wTrue = tf.reshape( allTrueWs[i], [ ] )
     uTrue = tf.reshape( tf.reverse(groundTruthU[i], [0]), [-1, 1] )

     if BenchParams:
          zTrue = tf.constant([0.01, 1.])
          wTrue =  tf.constant([2.])
     
     residual  = tf_residualFunction( tf.concat([ tf.squeeze(zTrue), wTrue[None]], 0) )
     u_untrans =  tf_fwNet( zTrue, wTrue, xttest_rav ) 

     u = tf.squeeze( initU(xtest, wTrue) + D_x(xtest) * D_t(ttest) * u_untrans )

     if PLOT:
          plt.rcParams.update({
          "text.usetex": True,
          "font.family": "sans-serif",
          "font.sans-serif": ["Helvetica"],
          'axes.labelsize': 16,
          'axes.titlesize': 16,
          'xtick.labelsize' : 16,
          'ytick.labelsize' : 16
                    })
          # latex font definition
          plt.rc('text', usetex=True)
          plt.rc('font', **{'family':'serif','serif':['Computer Modern Roman']})
          
          plt.contourf(Xtest, Ttest, tf.reshape(u, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.colorbar()
          plt.tight_layout()
          plt.savefig(dirname+'/'+'uSample_{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'uSample_{}.png'.format(i), u)
          plt.show() 


          plt.contourf(Xtest, Ttest, tf.reshape(u_untrans, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.colorbar()
          plt.tight_layout()
          plt.savefig(dirname+'/'+'u_untrans{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'u_untrans{}.png'.format(i), u_untrans)
          plt.show() 

          plt.contourf(Xtest, Ttest, tf.reshape(uTrue, [nx, nt]), 50)
          plt.ylabel(r'$t$')
          plt.xlabel(r'$x$')
          plt.tight_layout()
          plt.colorbar()
          plt.savefig(dirname+'/'+'uTrue{}.png'.format(i))
          if savePlots: np.save(dirname+'/'+'uTrue{}.png'.format(i), uTrue)
          plt.show()  


     u_true_unstranf = (uTrue - initU(xtest, wTrue)) / ( D_x(xtest) * D_t(ttest) )

     interpedU = kip.call(xttest_rav, tf.reshape(u_true_unstranf, [-1,1]), gridXTr)

     z = tf_bwNet( tf.reshape(interpedU, [-1,1]), wTrue,  gridXTr )
     z = tf.squeeze( z )

     print('true z = '   , zTrue)
     print('true w = '   , wTrue)
     print('\n\n')


     if saveDataList:

          allresAbsMean.append( tf.squeeze(residual) ) 

          allUPreds.append( tf.squeeze(u) )

          allPredZs.append( tf.squeeze(z) )
          
          allZNSE.append( ( tf.linalg.norm((tf.squeeze(z) - tf.squeeze(zTrue)) ) / tf.linalg.norm(tf.squeeze(zTrue)) )**2. )

          allUNSE.append( ( tf.linalg.norm((tf.squeeze(u) - tf.squeeze(uTrue)) ) / tf.linalg.norm(tf.squeeze(uTrue)) )**2. )

print('mean allZNSE = ', tf.reduce_mean( tf.stack(allZNSE) ) )
print('mean allUNSE = ', tf.reduce_mean( tf.stack(allUNSE) ) )

print('sttdev allZNSE = ', tf.math.reduce_std( tf.stack(allZNSE) ) )
print('sttdev allUNSE = ', tf.math.reduce_std( tf.stack(allUNSE) ) )


if writeToFile:
     with  open(dirname+"allInfoRun.txt", "w") as f:
          f.write('mean allZNSE = {}\n'.format(tf.reduce_mean( tf.stack(allZNSE) )) )
          f.write('mean allUNSE = {}\n'.format(tf.reduce_mean( tf.stack(allUNSE) )) )
          f.write('sttdev allZNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allZNSE) )) )
          f.write('sttdev allUNSE = {}\n'.format(tf.math.reduce_std( tf.stack(allUNSE) )) )
          f.write('totalTimeTrainingPDLVM = {:.2f}m\n'.format( timeTrainfw + timeTrainbw ))
     f.close()

print('totalTimeTrainingPDLVM = {:.2f}m'.format( timeTrainfw + timeTrainbw ) )



raise

In [ ]:
if True:
    alphaNet.save_weights(dirname + '/alphaModel/alphaModel-lfmDiag')
    # alphaNet.load_weights(dirname + '/alphaModel/alphaModel-lfmDiag')

    # del alphaNet

if True:
    betaNet.save_weights(dirname + '/betaModel/betaModel-lfmDiag')
    # betaNet.load_weights(dirname + '/betaModel/betaModel-lfmDiag')

    # del betaNet


In [ ]:
saveData = True
if saveData:

    allSampleZs_np = tf.stack(allSampleZs)
    allMeanZs_np   = tf.stack(allMeanZs)
    allStdZs_np    = tf.stack(allStdZs)

    allUSamples_np  = tf.stack(allUSamples)
    allUMeans_np    = tf.stack(allUMeans)
    allUStd_np      = tf.stack(allUStd)

    prefix = 'randgrid{}_{}kIter'.format(nxSamples, int(num_iterations/1000))

    np.savetxt(dirname + prefix + '_allRes_np.dat'   , tf.stack(tf.squeeze(allresAbsMean)) )


    np.savetxt(dirname + prefix + '_allSampleZs_np.dat', allSampleZs_np)
    np.savetxt(dirname + prefix + '_allMeanZs_np.dat'  , allMeanZs_np)
    np.savetxt(dirname + prefix + '_allStdZs_np.dat'  , allMeanZs_np)


    np.savetxt(dirname + prefix + '_allUSamples_np.dat' , allUSamples_np)
    np.savetxt(dirname + prefix + '_allUMeans_np.dat'   , allUMeans_np)
    np.savetxt(dirname + prefix + '_allUStd_np.dat'     , allUStd_np)
    
    np.savetxt(dirname + prefix + '_allZNSE_np.dat'   , tf.stack( allZNSE ) )
    np.savetxt(dirname + prefix + '_allUNSE_np.dat'   , tf.stack( allUNSE ) )

In [ ]:
for layer in betaNet.NN.kipLayers:
    print(layer.ell)